# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
df=pd.read_csv('cities_data.csv')
df.dropna(inplace=True)
df

,city,lat,long,temp,humidity,cloudiness,windspeed,country,date
0,kapaa,22.0752,-159.3190,70.79,73.0,40.0,20.71,US,1.621177e+09
1,panguna,-6.3164,155.4848,68.63,94.0,100.0,2.80,PG,1.621177e+09
2,saskylakh,71.9167,114.0833,17.87,93.0,100.0,16.71,RU,1.621177e+09
3,bredasdorp,-34.5322,20.0403,60.80,63.0,51.0,10.36,ZA,1.621177e+09
4,ostrovnoy,68.0531,39.5131,34.97,86.0,100.0,10.00,RU,1.621177e+09
...,...,...,...,...,...,...,...,...,...
612,tavda,58.0425,65.2726,74.39,40.0,53.0,6.85,RU,1.621180e+09
613,matagami,49.7502,-77.6328,64.40,45.0,20.0,10.36,CA,1.621180e+09
614,yaan,7.3833,8.5667,88.29,56.0,84.0,7.96,NG,1.621180e+09
615,rudnya,54.9470,31.0934,71.60,43.0,20.0,6.71,RU,1.621180e+09


In [6]:
map_df=df[["lat", "long", "humidity"]]
map_df


,lat,long,humidity
0,22.0752,-159.3190,73.0
1,-6.3164,155.4848,94.0
2,71.9167,114.0833,93.0
3,-34.5322,20.0403,63.0
4,68.0531,39.5131,86.0
...,...,...,...
612,58.0425,65.2726,40.0
613,49.7502,-77.6328,45.0
614,7.3833,8.5667,56.0
615,54.9470,31.0934,43.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [81]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = map_df[['lat', 'long']].astype(float)
humidity = map_df['humidity'].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
df
hotel_df=df[(df['temp'] < 80) & (df['temp'] > 70)]
hotel_df=hotel_df[(hotel_df['windspeed'] < 10) & (hotel_df['cloudiness'] == 0)]
hotel_df


,city,lat,long,temp,humidity,cloudiness,windspeed,country,date
26,saint-philippe,-21.3585,55.7679,74.73,73.0,0.0,3.44,RE,1.621177e+09
84,saint-joseph,-21.3667,55.6167,74.57,73.0,0.0,3.44,RE,1.621177e+09
114,saint-leu,-21.1500,55.2833,74.77,73.0,0.0,3.44,RE,1.621177e+09
180,puerto penasco,31.3167,-113.5333,73.00,63.0,0.0,5.99,MX,1.621178e+09
195,trojes,41.7560,19.9620,71.60,46.0,0.0,5.75,AL,1.621178e+09
230,zheleznodorozhnyy,55.7503,38.0042,73.47,43.0,0.0,2.24,RU,1.621178e+09
326,paraiso,9.8383,-83.8656,70.70,68.0,0.0,4.52,CR,1.621178e+09
408,saint-pierre,-21.3393,55.4781,74.10,73.0,0.0,2.30,RE,1.621179e+09
433,guacimo,10.2104,-83.6807,70.57,68.0,0.0,4.25,CR,1.621179e+09
473,buckeye,33.3703,-112.5838,76.26,31.0,0.0,5.75,US,1.621179e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
hotel_df["Hotel Name"]=""
hotel_df
hotel_df_short=hotel_df.reset_index(drop=True)
hotel_df_short


,city,lat,long,temp,humidity,cloudiness,windspeed,country,date,Hotel Name
0,saint-philippe,-21.3585,55.7679,74.73,73.0,0.0,3.44,RE,1.621177e+09,
1,saint-joseph,-21.3667,55.6167,74.57,73.0,0.0,3.44,RE,1.621177e+09,
2,saint-leu,-21.1500,55.2833,74.77,73.0,0.0,3.44,RE,1.621177e+09,
3,puerto penasco,31.3167,-113.5333,73.00,63.0,0.0,5.99,MX,1.621178e+09,
4,trojes,41.7560,19.9620,71.60,46.0,0.0,5.75,AL,1.621178e+09,
5,zheleznodorozhnyy,55.7503,38.0042,73.47,43.0,0.0,2.24,RU,1.621178e+09,
6,paraiso,9.8383,-83.8656,70.70,68.0,0.0,4.52,CR,1.621178e+09,
7,saint-pierre,-21.3393,55.4781,74.10,73.0,0.0,2.30,RE,1.621179e+09,
8,guacimo,10.2104,-83.6807,70.57,68.0,0.0,4.25,CR,1.621179e+09,
9,buckeye,33.3703,-112.5838,76.26,31.0,0.0,5.75,US,1.621179e+09,


In [38]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key
}

In [68]:
for i, row in hotel_df_short.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["lat"]
    lng = row["long"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_name = response['results'][1]['name']
        hotel_df_short.loc[i, "Hotel Name"] = hotel_name

    except IndexError:
        print(hotel_df_short.loc[i, 'city'])
        print(response)
        

hotel_df_short

trojes
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


,city,lat,long,temp,humidity,cloudiness,windspeed,country,date,Hotel Name
0,saint-philippe,-21.3585,55.7679,74.73,73.0,0.0,3.44,RE,1.621177e+09,Le four à pain
1,saint-joseph,-21.3667,55.6167,74.57,73.0,0.0,3.44,RE,1.621177e+09,La Case O zoizos
2,saint-leu,-21.1500,55.2833,74.77,73.0,0.0,3.44,RE,1.621177e+09,Blue Margouillat
3,puerto penasco,31.3167,-113.5333,73.00,63.0,0.0,5.99,MX,1.621178e+09,Best Western Laos Mar Hotel and Suites
4,trojes,41.7560,19.9620,71.60,46.0,0.0,5.75,AL,1.621178e+09,
5,zheleznodorozhnyy,55.7503,38.0042,73.47,43.0,0.0,2.24,RU,1.621178e+09,Hotel Dali
6,paraiso,9.8383,-83.8656,70.70,68.0,0.0,4.52,CR,1.621178e+09,Paraiso Orocay Lodge
7,saint-pierre,-21.3393,55.4781,74.10,73.0,0.0,2.30,RE,1.621179e+09,Le Battant Des Lames
8,guacimo,10.2104,-83.6807,70.57,68.0,0.0,4.25,CR,1.621179e+09,Hotel Los Tabacones
9,buckeye,33.3703,-112.5838,76.26,31.0,0.0,5.75,US,1.621179e+09,Westward Motel


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_short.iterrows()]
locations = hotel_df_short[["lat", "long"]]

In [80]:
# Add marker layer ontop of heat map
fig_one=fig
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_one.add_layer(markers)
fig_one
# Display figure


Figure(layout=FigureLayout(height='420px'))